### MLBD

In [16]:
# !pip install --upgrade pyspark
# !pip install --upgrade matplotlib==3.6.1

In [415]:
# !pip install sklearn numpy pandas datasist
# !pip install pyspark spark spark-nlp
# !pip install geopandas shapely
# !pip install matplotlib mpl_toolkits
# !pip install plotly kaleido
# !pip install -U kaleido
#!pip install folium
#!pip install geopy

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from functools import reduce  # For Python 3.x
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Row
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
import glob
import folium
import warnings
import geopy

In [2]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
spark = SparkSession.builder.appName('airscholar').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/13 14:30:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [701]:
# spark._sc.stop()

In [13]:
def readAllJSON():
    """
    This functions reads all the datasets downloaded into datasets folder
    """
    spark.sparkContext.addFile('datasets', recursive=True)
    jsons = glob.glob("datasets/*.json")
    jsons.sort()
    base_data_df = []
    for i in range(2, 4):
        json_obj = jsons[i]
        t_df = spark.read.json("file://"+SparkFiles.get(json_obj))
        t_df = t_df.sort(t_df.timestamp)
        # t_df.show()
        base_data_df.append(t_df)
    return base_data_df

def extractSensor(base_data_df):
    """
    This function extracts the values in the sensor field
    """
    base_data_df = base_data_df.withColumn('timestamp', base_data_df.timestamp.cast(TimestampType()))
    sensor_df = base_data_df\
    .withColumn('manufacturer', base_data_df.sensor.sensor_type.getField('manufacturer'))\
    .withColumn('sensorId', base_data_df.sensor.getField('id').cast(StringType()))\
    .withColumn('sdvID', base_data_df.sensordatavalues.getField('id').cast(StringType()))\
    .withColumn('sdvValue', base_data_df.sensordatavalues.getField('value'))\
    .withColumn('sdvValue_type', base_data_df.sensordatavalues.getField('value_type'))
    sensor_df = sensor_df.drop('sampling_rate', 'sensor', 'sensordatavalues')
    sesor_df = sensor_df.sort(sensor_df.sensorId)
    sensor_df = sensor_df.filter(array_contains(sensor_df['sdvValue_type'], 'P1') & array_contains(sensor_df['sdvValue_type'], 'P2'))

    sensor_df = sensor_df.withColumn('P1_Pos', array_position(sensor_df.sdvValue_type, "P1")-1).withColumn('P2_Pos', array_position(sensor_df.sdvValue_type, "P2")-1)
    sensor_df = sensor_df.withColumn('P1', sensor_df['sdvValue'][sensor_df.P1_Pos.cast(IntegerType())])\
    .withColumn('P2', sensor_df['sdvValue'][sensor_df.P2_Pos.cast(IntegerType())])

    sensor_df.withColumn('P1', sensor_df.P1.cast(FloatType())).withColumn('P2', sensor_df.P2.cast(FloatType()))
    
    sensor_df = sensor_df.withColumn('country', base_data_df.location.getField('country'))\
    .withColumn('latitude', sensor_df.location.getField('latitude').cast(FloatType()))\
    .withColumn('longitude', sensor_df.location.getField('longitude').cast(FloatType())).drop('location')
    
    sensor_df = sensor_df.drop('sdvID','sdvValue_type', 'sdvValue', 'P1_Pos', 'P2_Pos')
    sensor_df = sensor_df[(sensor_df.latitude != 0) & (sensor_df.longitude != 0)]
    
    return sensor_df

def generate_model(df):
    """
    This function generates the model of the first day which is used for the prediction of other days
    """
    vecAssembler = VectorAssembler(inputCols=['latitude', 'longitude'], outputCol="features")
    X = df.select('id', 'latitude', 'longitude')
    new_df = vecAssembler.transform(X)
    kmeans = KMeans(k=100, maxIter=10, seed=1)
    model = kmeans.fit(new_df)
    
    return model

def compute_predictions(model, df):    
    """
    This function uses the generated model of the first day to predict each of the other days
    """
    vecAssembler = VectorAssembler(inputCols=['latitude', 'longitude'], outputCol="features")
    x = df.select('id', 'latitude', 'longitude')
    vec = vecAssembler.transform(x)
    prediction = model.transform(vec)
    
    prediction = prediction.drop('features','latitude', 'longitude')
    df = df.join(prediction, ['id'], 'inner')
    
    return df

def generate_cluster(model, df):
    """
    This function generates the cluster center for each of the predictions
    """
    XX = pd.DataFrame(model.clusterCenters(), columns=['longitude', 'latitude'])
    XX['cluster_index'] = XX.index.map(lambda x: format(x))
    XX = spark.createDataFrame(XX)
    XX = XX.withColumn('cluster_center', concat(array(col('latitude'), col('latitude')))).drop('longitude', 'latitude')
    
    df = df.join(XX, df.prediction == XX.cluster_index, 'inner')
    df = df.drop('cluster_index')
    
    return df

def mergedTwoDates(df1, df2):
    """
    This function merges two dates together
    """
    merged = df1.withColumnRenamed('P1', 'P1_1').withColumnRenamed('P2', 'P2_1')\
        .withColumnRenamed('timestamp', 'timestamp_1')\
        .withColumnRenamed('country', 'country_1')\
        .withColumnRenamed('latitude', 'latitude_1')\
        .withColumnRenamed('longitude', 'longitude_1')\
        .withColumnRenamed('prediction', 'prediction_1')\
        .withColumnRenamed('cluster_center', 'cluster_center_1')\
        .join(df2.withColumnRenamed('P1', 'P1_2').withColumnRenamed('P2', 'P2_2')\
        .withColumnRenamed('timestamp', 'timestamp_2')
        .withColumnRenamed('country', 'country_2')\
        .withColumnRenamed('latitude', 'latitude_2')\
        .withColumnRenamed('longitude', 'longitude_2')\
        .withColumnRenamed('prediction', 'prediction_2')\
        .withColumnRenamed('cluster_center', 'cluster_center_2'), 
          ['sensorId'], 'fullouter')
    merged = merged.withColumn('P1_1', merged.P1_1.cast(FloatType())).withColumn('P1_2', merged.P1_2.cast(FloatType()))\
        .withColumn('P2_1', merged.P2_1.cast(FloatType())).withColumn('P2_2', merged.P2_2.cast(FloatType())).drop('id', 'manufacturer')
    merged = merged.na.fill(value=0, subset=['P1_1', 'P1_2', 'P2_1', 'P2_2'])
    merged = merged.withColumn('P1', round((merged.P1_1+merged.P1_2)/2)).withColumn('P2', round((merged.P2_1+merged.P2_2)/2))#.drop('P1_1', 'P2_1', 'P1_2', 'P2_2')
    merged = merged.withColumn('country', coalesce(merged.country_1, merged.country_2))\
        .withColumn('latitude', coalesce(merged.latitude_1, merged.latitude_2))\
        .withColumn('longitude', coalesce(merged.longitude_1, merged.longitude_2))\
        .withColumn('prediction', coalesce(merged.prediction_1, merged.prediction_2))\
        .withColumn('cluster_center', coalesce(merged.cluster_center_1, merged.cluster_center_2))
                    
    merged = merged.drop('country_1', 'latitude_1', 'longitude_1', 'cluster_center_1', 'cluster_center_2', 'country_2', 'latitude_2', 'longitude_2', 'prediction_1', 'prediction_2')
    return merged

def generate_AQI(df):
    """
    This function generates the Air Quality Indeces of the each of the prediction
    """
    df = df.withColumn('Day1_AQI', \
                     when((df.P1 >= 0) & (df.P1 <= 16), lit(1))\
                     .when((df.P1 >= 17) & (df.P1 <= 33), lit(2))\
                     .when((df.P1 >= 34) & (df.P1 <= 50), lit(3))\
                     .when((df.P1 >= 51) & (df.P1 <= 58), lit(4))\
                     .when((df.P1 >= 59) & (df.P1 <= 66), lit(5))\
                     .when((df.P1 >= 67) & (df.P1 <= 75), lit(6))\
                     .when((df.P1 >= 76) & (df.P1 <= 83), lit(7))\
                     .when((df.P1 >= 84) & (df.P1 <= 91), lit(8))\
                     .when((df.P1 >= 92) & (df.P1 <= 100), lit(9))\
                     .when((df.P1 > 100), lit(10))\
                    )
    df = df.withColumn('Day2_AQI', \
                         when((df.P2 >= 0) & (df.P2 <= 11), lit(1))\
                         .when((df.P2 >= 12) & (df.P2 <= 23), lit(2))\
                         .when((df.P2 >= 24) & (df.P2 <= 35), lit(3))\
                         .when((df.P2 >= 36) & (df.P2 <= 41), lit(4))\
                         .when((df.P2 >= 42) & (df.P2 <= 47), lit(5))\
                         .when((df.P2 >= 48) & (df.P2 <= 53), lit(6))\
                         .when((df.P2 >= 54) & (df.P2 <= 58), lit(7))\
                         .when((df.P2 >= 59) & (df.P2 <= 64), lit(8))\
                         .when((df.P2 >= 65) & (df.P2 <= 69), lit(9))\
                         .when((df.P2 > 70), lit(10))\
                        )
    df = df.withColumn('AQI', when((df.Day1_AQI < df.Day2_AQI), lit(df.Day2_AQI)).otherwise(lit(df.Day1_AQI)))
    df = df.withColumn('AQI_Range', \
                         when((df.AQI >= 1) & (df.AQI <= 3), lit('Low'))\
                         .when((df.AQI >= 4) & (df.AQI <= 6), lit('Medium'))\
                         .when((df.AQI >= 7) & (df.AQI <= 9), lit('High'))\
                         .when((df.AQI == 10), lit('Very High'))\
                        )
    df = df.withColumn('AQI_Improv', lit(df.Day2_AQI - df.Day1_AQI))
    return df


In [14]:
base_df = readAllJSON()

22/11/13 14:32:02 WARN SparkContext: The path datasets has been added already. Overwriting of added paths is not supported in the current version.
+-----------+--------------------+-------------+--------------------+--------------------+-------------------+
|         id|            location|sampling_rate|              sensor|    sensordatavalues|          timestamp|
+-----------+--------------------+-------------+--------------------+--------------------+-------------------+
|12738558804|{563.8, BG, 0, 19...|         null|{33155, 11, {17, ...|[{28444726678, 10...|2022-10-29 09:06:52|
|12738573344|{36.1, DE, 0, 155...|         null|{28245, 1, {14, N...|[{28444759869, 12...|2022-10-29 09:09:46|
|12738589682|{36.1, DE, 0, 155...|         null|{28244, 7, {9, va...|[{28444797266, 99...|2022-10-29 09:12:48|
|12738591761|{102.6, DE, 1, 31...|         null|{45998, 1, {14, N...|[{28444801993, 9....|2022-10-29 09:13:16|
|12738599503|{123.2, PL, 0, 22...|         null|{35847, 1, {14, N...|[{28444

In [15]:
arrs = []
for df in base_df:
    arrs.append(extractSensor(df))

### generate a model object using the first day

In [16]:
model = generate_model(arrs[0])

22/11/13 14:32:19 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


### generate predictions for the other days

In [17]:
preds = []
for i in range(0, len(arrs)):
    pred = compute_predictions(model, arrs[i])
    pred = generate_cluster(model, pred)
    preds.append(pred)

In [18]:
merged = mergedTwoDates(preds[0], preds[1])

In [22]:
merged.show(truncate=False)

+--------+-------------------+-----+-----+-------------------+-----+-----+----+----+-------+---------+---------+----------+----------------------------------------+
|sensorId|timestamp_1        |P1_1 |P2_1 |timestamp_2        |P1_2 |P2_2 |P1  |P2  |country|latitude |longitude|prediction|cluster_center                          |
+--------+-------------------+-----+-----+-------------------+-----+-----+----+----+-------+---------+---------+----------+----------------------------------------+
|1000    |2022-10-30 09:03:29|8.77 |4.84 |2022-10-30 12:09:43|8.89 |4.9  |9.0 |5.0 |DE     |47.806   |12.858   |29        |[11.72619804982786, 11.72619804982786]  |
|10001   |2022-10-30 09:04:38|15.59|11.51|2022-10-30 12:05:26|15.72|11.68|16.0|12.0|BG     |42.669796|23.2684  |85        |[23.337626838289186, 23.337626838289186]|
|10005   |2022-10-30 09:04:56|7.42 |4.96 |2022-10-30 12:09:51|7.5  |5.06 |7.0 |5.0 |BG     |42.626   |23.382   |85        |[23.337626838289186, 23.337626838289186]|
|10007   |

In [23]:
merged = generate_AQI(merged)

In [24]:
merged.show()

+--------+-------------------+-----+-----+-------------------+-----+-----+----+----+-------+---------+---------+----------+--------------------+--------+--------+---+---------+----------+
|sensorId|        timestamp_1| P1_1| P2_1|        timestamp_2| P1_2| P2_2|  P1|  P2|country| latitude|longitude|prediction|      cluster_center|Day1_AQI|Day2_AQI|AQI|AQI_Range|AQI_Improv|
+--------+-------------------+-----+-----+-------------------+-----+-----+----+----+-------+---------+---------+----------+--------------------+--------+--------+---+---------+----------+
|    1000|2022-10-30 09:03:29| 8.77| 4.84|2022-10-30 12:09:43| 8.89|  4.9| 9.0| 5.0|     DE|   47.806|   12.858|        29|[11.7261980498278...|       1|       1|  1|      Low|         0|
|   10001|2022-10-30 09:04:38|15.59|11.51|2022-10-30 12:05:26|15.72|11.68|16.0|12.0|     BG|42.669796|  23.2684|        85|[23.3376268382891...|       1|       2|  2|      Low|         1|
|   10005|2022-10-30 09:04:56| 7.42| 4.96|2022-10-30 12:09:5

In [25]:
impr = merged.sort(merged.AQI_Improv.desc())

In [26]:
impr.show()

+--------+-------------------+-----+------+-------------------+------+------+----+-----+-------+---------+---------+----------+--------------------+--------+--------+---+---------+----------+
|sensorId|        timestamp_1| P1_1|  P2_1|        timestamp_2|  P1_2|  P2_2|  P1|   P2|country| latitude|longitude|prediction|      cluster_center|Day1_AQI|Day2_AQI|AQI|AQI_Range|AQI_Improv|
+--------+-------------------+-----+------+-------------------+------+------+----+-----+-------+---------+---------+----------+--------------------+--------+--------+---+---------+----------+
|   54458|2022-10-30 09:04:53|43.14|131.04|2022-10-30 12:09:53| 44.54|133.18|44.0|132.0|     DE|    49.35|    8.152|        16|[8.52343586364888...|       3|      10| 10|Very High|         7|
|   16147|2022-10-30 09:05:03|76.12| 73.45|2022-10-30 12:08:33| 70.05| 67.63|73.0| 71.0|     SE|   59.364|   18.018|        22|[17.8251520477899...|       6|      10| 10|Very High|         4|
|   13609|2022-10-30 09:04:25|46.93| 42.

In [29]:
impr.select('country', 'AQI_Improv').where(impr.AQI_Improv > 0).groupBy(impr.country, impr.AQI_Improv).count().sort(impr.country.asc(), impr.AQI_Improv.desc()).show(100)

+-------+----------+-----+
|country|AQI_Improv|count|
+-------+----------+-----+
|     AT|         2|    3|
|     AT|         1|    4|
|     BE|         3|    1|
|     BE|         2|    1|
|     BE|         1|    4|
|     BG|         2|    2|
|     BG|         1|   14|
|     BY|         1|    3|
|     CA|         1|    1|
|     CH|         1|    1|
|     CZ|         2|    2|
|     CZ|         1|    7|
|     DE|         7|    1|
|     DE|         2|    9|
|     DE|         1|   64|
|     ES|         2|    1|
|     ES|         1|    1|
|     FR|         2|    2|
|     FR|         1|   29|
|     GB|         1|    9|
|     GR|         1|    3|
|     HR|         1|    1|
|     HU|         2|    1|
|     HU|         1|    6|
|     IE|         3|    1|
|     IE|         2|    1|
|     IT|         1|    3|
|     KG|         1|    1|
|     NL|         3|    1|
|     NL|         2|    3|
|     NL|         1|   38|
|     NO|         1|    1|
|     PL|         3|    1|
|     PL|         2|    3|
|

In [ ]:
plt.rcParams['figure.figsize'] = [16, 10]

In [30]:
m = folium.Map(zoom_start=10, location=model.clusterCenters()[0], tiles="Stamen Terrain")

for location in model.clusterCenters()[0:50]:
    folium.Marker(location.tolist()).add_to(m)

m

In [ ]:
import jovian
jovian.commit(filename='MLBD.ipynb')